#Installs & Imports

In [1]:
!pip install transformers
!pip install torch
!pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Load data
df = pd.read_csv('/content/drive/MyDrive/data_sets/compas-scores-two-years.csv')

In [5]:
df = df[(df["days_b_screening_arrest"] <= 30)
        & (df["days_b_screening_arrest"] >= -30)
        & (df["is_recid"] != -1)
        & (df["c_charge_degree"] != 'O')
        & (df["score_text"] != 'N/A')].reset_index(drop=True)

In [6]:
cols_to_keep = ['sex','age', 'race', 'juv_fel_count', 'juv_misd_count', 'decile_score', 'juv_other_count', 'priors_count',
                'c_charge_degree', 'two_year_recid']

df = df[cols_to_keep]
df

,sex,age,race,juv_fel_count,juv_misd_count,decile_score,juv_other_count,priors_count,c_charge_degree,two_year_recid
0,Male,69,Other,0,0,1,0,0,F,0
1,Male,34,African-American,0,0,3,0,0,F,1
2,Male,24,African-American,0,0,4,1,4,F,1
3,Male,44,Other,0,0,1,0,0,M,0
4,Male,41,Caucasian,0,0,6,0,14,F,1
...,...,...,...,...,...,...,...,...,...,...
6167,Male,23,African-American,0,0,7,0,0,F,0
6168,Male,23,African-American,0,0,3,0,0,F,0
6169,Male,57,Other,0,0,1,0,0,F,0
6170,Female,33,African-American,0,0,2,0,3,M,0


In [7]:
df.dtypes

,0
sex,object
age,int64
race,object
juv_fel_count,int64
juv_misd_count,int64
decile_score,int64
juv_other_count,int64
priors_count,int64
c_charge_degree,object
two_year_recid,int64


In [8]:
#Number of missing values in the df
df.isna().sum()

,0
sex,0
age,0
race,0
juv_fel_count,0
juv_misd_count,0
decile_score,0
juv_other_count,0
priors_count,0
c_charge_degree,0
two_year_recid,0


In [9]:
#Filter df to include only Caucasian and African American
df = df[df['race'].isin(['Caucasian', 'African-American'])]
df

,sex,age,race,juv_fel_count,juv_misd_count,decile_score,juv_other_count,priors_count,c_charge_degree,two_year_recid
1,Male,34,African-American,0,0,3,0,0,F,1
2,Male,24,African-American,0,0,4,1,4,F,1
4,Male,41,Caucasian,0,0,6,0,14,F,1
6,Female,39,Caucasian,0,0,1,0,0,M,0
7,Male,27,Caucasian,0,0,4,0,0,F,0
...,...,...,...,...,...,...,...,...,...,...
6165,Male,30,African-American,0,0,2,0,0,M,1
6166,Male,20,African-American,0,0,9,0,0,F,0
6167,Male,23,African-American,0,0,7,0,0,F,0
6168,Male,23,African-American,0,0,3,0,0,F,0


#Splitting

In [10]:
#Combining features into a single string for BERT input
df['text'] = df.apply(lambda row: f"Sex: {row['sex']}, Age: {row['age']}, Race: {row['race']}, Felony count: {row['juv_fel_count']}, Misdemeanor count: {row['juv_misd_count']}, Decile score: {row['decile_score']}, Prior count: {row['priors_count']}, Charge degree: {row['c_charge_degree']}", axis=1)

#Target variable
df['label'] = df['two_year_recid']

#Split data into train (70%), validation (15%), and test (15%) sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

<ipython-input-10-3e2947cf12a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df.apply(lambda row: f"Sex: {row['sex']}, Age: {row['age']}, Race: {row['race']}, Felony count: {row['juv_fel_count']}, Misdemeanor count: {row['juv_misd_count']}, Decile score: {row['decile_score']}, Prior count: {row['priors_count']}, Charge degree: {row['c_charge_degree']}", axis=1)
<ipython-input-10-3e2947cf12a7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['two_year_recid']


In [11]:
from transformers import BertTokenizer

#BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#Tokenize data
def tokenize_data(df):
    return tokenizer(
        df['text'].tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)
test_encodings = tokenize_data(test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
import torch

class RecidivismDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = RecidivismDataset(train_encodings, train_df['label'].tolist())
val_dataset = RecidivismDataset(val_encodings, val_df['label'].tolist())
test_dataset = RecidivismDataset(test_encodings, test_df['label'].tolist())

In [13]:
#Evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  #index of the max logit as the predicted class

    #Calculate accuracy score
    accuracy = accuracy_score(labels, preds)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

#Training BERT model

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [14]:
#Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

#Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#Train the model
trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-ef781c7b9d23>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.650817,0.651515,0.701245,0.453083,0.550489
2,No log,0.643863,0.655303,0.612613,0.729223,0.665851
3,0.659900,0.612754,0.665404,0.654286,0.613941,0.633472


<ipython-input-12-ef781c7b9d23>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-ef781c7b9d23>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-ef781c7b9d23>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=693, training_loss=0.6483682849878528, metrics={'train_runtime': 48.3629, 'train_samples_per_second': 229.143, 'train_steps_per_second': 14.329, 'total_flos': 250576280238240.0, 'train_loss': 0.6483682849878528, 'epoch': 3.0})

#Model performance metrics

In [15]:
#Evaluate on validation set
val_results = trainer.evaluate(eval_dataset=val_dataset)
print("Validation Metrics:")
print(f"Accuracy: {val_results['eval_accuracy']:.4f}")
print(f"Precision: {val_results['eval_precision']:.4f}")
print(f"Recall: {val_results['eval_recall']:.4f}")
print(f"F1 Score: {val_results['eval_f1']:.4f}")

#Evaluate on test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Metrics:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Precision: {test_results['eval_precision']:.4f}")
print(f"Recall: {test_results['eval_recall']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

<ipython-input-12-ef781c7b9d23>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Validation Metrics:
Accuracy: 0.6654
Precision: 0.6543
Recall: 0.6139
F1 Score: 0.6335


<ipython-input-12-ef781c7b9d23>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Test Metrics:
Accuracy: 0.6995
Precision: 0.6982
Recall: 0.6344
F1 Score: 0.6648


In [16]:
#Predict on test set
predictions = trainer.predict(test_dataset)

<ipython-input-12-ef781c7b9d23>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [17]:
!pip install fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.1/234.1 kB 5.1 MB/s eta 0:00:00


#Fairness Metrics

In [18]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score, precision_score
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference, true_positive_rate_difference, false_positive_rate_difference

In [19]:
#predictions and true labels
y_pred = predictions.predictions.argmax(axis=1)
y_true = predictions.label_ids

In [20]:
#Sensitive features for fairness evaluation
sensitive_features_race = test_df['race']
sensitive_features_sex = test_df['sex']

In [21]:
#Function to calculate PPV (Precision) for each group
def calculate_ppv(y_true, y_pred, sensitive_features):
    unique_groups = np.unique(sensitive_features)
    ppv_per_group = {}
    for group in unique_groups:
        indices = (sensitive_features == group)
        y_true_group = y_true[indices]
        y_pred_group = y_pred[indices]
        # Calculate PPV for this group
        ppv = precision_score(y_true_group, y_pred_group, pos_label=1)
        ppv_per_group[group] = ppv
    return ppv_per_group

####Fairness Metrics - Race

In [22]:
#Acc per group
metric_frame = MetricFrame(
    metrics=accuracy_score,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features_race
)

#Fairness metrics - race
dp_diff = demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_features_race)
print("Demographic Parity Difference:", dp_diff)

eo_diff = equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_features_race)
print("Equalized Odds Difference:", eo_diff)

tpr_diff = true_positive_rate_difference(y_true, y_pred, sensitive_features=sensitive_features_race)
print("TPR Difference:", tpr_diff)

fpr_diff = false_positive_rate_difference(y_true, y_pred, sensitive_features=sensitive_features_race)
print("FPR Difference:", fpr_diff)

Demographic Parity Difference: 0.2593384250449652
Equalized Odds Difference: 0.3045862412761715
TPR Difference: 0.3045862412761715
FPR Difference: 0.14360977382141885


In [23]:
#Calculate PPV for each group based on race
ppv_per_group_race = calculate_ppv(y_true, y_pred, sensitive_features_race)
print("PPV per group (Race):")
for group, ppv in ppv_per_group_race.items():
    print(f"Group {group}: PPV = {ppv}")

#Predictive Parity Difference - race
group_list_race = list(ppv_per_group_race.keys())
ppv_diff_race = ppv_per_group_race[group_list_race[0]] - ppv_per_group_race[group_list_race[1]]
print("Predictive Parity Difference (Race):", ppv_diff_race)

PPV per group (Race):
Group African-American: PPV = 0.7213114754098361
Group Caucasian: PPV = 0.6382978723404256
Predictive Parity Difference (Race): 0.08301360306941052


####Fairness Metrics - Sex

In [24]:
metric_frame = MetricFrame(
    metrics=accuracy_score,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=sensitive_features_sex
)

#Fairness metrics - sex
dp_diff = demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_features_sex)
print("Demographic Parity Difference:", dp_diff)

eo_diff = equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_features_sex)
print("Equalized Odds Difference:", eo_diff)

tpr_diff = true_positive_rate_difference(y_true, y_pred, sensitive_features=sensitive_features_sex)
print("TPR Difference:", tpr_diff)

fpr_diff = false_positive_rate_difference(y_true, y_pred, sensitive_features=sensitive_features_sex)
print("FPR Difference:", fpr_diff)

Demographic Parity Difference: 0.30745525660779904
Equalized Odds Difference: 0.4094409937888198
TPR Difference: 0.4094409937888198
FPR Difference: 0.16000789188122716


In [25]:
# Calculate PPV for each group based on sex
ppv_per_group_sex = calculate_ppv(y_true, y_pred, sensitive_features_sex)
print("PPV per group (Sex):")
for group, ppv in ppv_per_group_sex.items():
    print(f"Group {group}: PPV = {ppv}")

#Predictive Parity Difference - sex
group_list_sex = list(ppv_per_group_sex.keys())
ppv_diff_sex = ppv_per_group_sex[group_list_sex[0]] - ppv_per_group_sex[group_list_sex[1]]
print("Predictive Parity Difference (Sex):", ppv_diff_sex)

PPV per group (Sex):
Group Female: PPV = 0.56
Group Male: PPV = 0.7092651757188498
Predictive Parity Difference (Sex): -0.14926517571884979
